# Workshop 1 - Question and Answers
In this workshop, you will learning how to write prompts and feed them into LLMs. You
will also be learning how to use different prompt techniques to improve the response
from the LLM.

## Loading and Explorng the Dataset
The workshop will be using [`facebook/ExploreToM`](https://huggingface.co/datasets/facebook/ExploreToM) dataset from [HuggingFace](https://huggingface.co).

In [1]:
# TODO: Load the following libraries: datasets
from datasets import load_dataset

In [2]:
# Dataset name
dataset_name = "facebook/ExploreToM"

# TODO: load and explore the dataset
ds = load_dataset(dataset_name)

Using the latest cached version of the dataset since facebook/ExploreToM couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/facebook___explore_to_m/default/0.0.0/a4ac6f257e0034945f829716047ae6306dc625a0 (last modified on Thu Sep  4 08:24:40 2025).


In [3]:
# TODO: number of rows in the dataset
## train dataset!
print(ds.shape)

# TODO: Keys in the dataset
print(ds.keys())

# TODO: Column names in the dataset
print(ds['train'].column_names)

# TODO: Feature names
print(ds['train'].features)


# TODO: Display a single row
idx = 100 # arbitrary index
for k,v in ds['train'][idx].items():
    print(f"key = {k}, value = {v}")

{'train': (13309, 18)}
dict_keys(['train'])
['story_structure', 'infilled_story', 'question', 'expected_answer', 'qprop=params', 'qprop=nth_order', 'qprop=non_unique_mental_state', 'sprop=is_false_belief_story_1st', 'sprop=is_false_belief_story_1st_and_2nd', 'sprop=story_accuracy_1st_raw', 'sprop=story_accuracy_1st_infilled', 'sprop=global_idx', 'param=story_type', 'param=num_stories_total', 'param=max_sentences', 'param=num_people', 'param=num_moves', 'param=num_rooms']
{'story_structure': Value('string'), 'infilled_story': Value('string'), 'question': Value('string'), 'expected_answer': Value('string'), 'qprop=params': Value('string'), 'qprop=nth_order': Value('int64'), 'qprop=non_unique_mental_state': Value('bool'), 'sprop=is_false_belief_story_1st': Value('bool'), 'sprop=is_false_belief_story_1st_and_2nd': Value('bool'), 'sprop=story_accuracy_1st_raw': Value('float64'), 'sprop=story_accuracy_1st_infilled': Value('float64'), 'sprop=global_idx': Value('int64'), 'param=story_type': Va

In [4]:
# TODO: import pipeline
from transformers import pipeline

## `pipeline`
[`pipeline`](https://huggingface.co/docs/transformers/en/main_classes/pipelines) is an easy to use API to perform inferencing. It provides a wrapper for task-specific pipelines and abstracts most of the complexity by allowing you to focus on the model and the task. 

You can use `pipeline` to perform summarisation, image classification, audio generation, etc. You can find an exhaustive list of `pipeline` task [here](https://huggingface.co/docs/transformers/en/main_classes/pipelines#transformers.pipeline.task).

In [5]:
# TODO: Summarise the text with the pipeline's default model
qna = pipeline("question-answering")
# can set specific model, e.g. model="distilbert-base-cased-distilled-squad"

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [6]:
# prepare question
idx = 100 # arbitrary index
question = ds['train'][idx]['question']
story = ds['train'][idx]['story_structure']
infilled_story = ds['train'][idx]['infilled_story']
expected_answer = ds['train'][idx]['expected_answer']

# chosen story
chosen_story = infilled_story

# query the model
# predicted_ans = qna(question=question, context=story)
predicted_ans = qna(question=question, context=chosen_story)

## Manual Inference - Question and Answer
In this section, we will look at what `pipeline` does under the hood to perform its inference. This will give us a better understanding of the major steps involved.

In [7]:
print(story)
print(chosen_story)
print('--------------------------------')
print(predicted_ans)
print(expected_answer)

Leslie entered the main tent. Leslie left the main tent. Isabella entered the storage trailer. Isabella moved the stuffed rabbit to the wooden chest, which is also located in the storage trailer. Leslie entered the main tent. Isabella moved the stuffed rabbit to the main tent, leaving the wooden chest in its original location. Isabella told out loud about the festival marketing strategies. Isabella told privately to Colton that Leslie is in the main tent. While this action was happening, Leslie witnessed this action in secret (and only this action).
The warm glow of string lights illuminated the vibrant colors of the festival grounds, casting a lively atmosphere over rows of booths and attractions. A faint scent of popcorn and sugar wafted through the air, mingling with the distant sounds of laughter and music, as the night's festivities had just begun to unfold. Leslie slipped unnoticed into the main tent, the sounds of the festival outside momentarily muffled by the canvas walls. Les

In [8]:
# TODO: load tokenizer
from transformers import AutoTokenizer

## DistilBERT base cased distilled SQuAD
DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. More details [here](https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).

In [9]:
model_name = "distilbert/distilbert-base-cased-distilled-squad"

In [10]:
# TODO: Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [11]:
# TODO: Encode text
## text >> tokenizer >> number 
### so that number can be fed to model
text = 'Thou shall not create a machine in the likeness of the human mind'

enc_text = tokenizer(text, return_tensors='pt')
print(enc_text)

{'input_ids': tensor([[  101,   157, 14640,  4103,  1136,  2561,   170,  3395,  1107,  1103,
          1176,  1757,  1104,  1103,  1769,  1713,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# number of words input != number of tokens
# just to get tensor input_ids, note that this is a 2D array
print(enc_text.input_ids[0].shape)

# to decode the encoded text
for i in enc_text.input_ids[0]:
    print(i, tokenizer.decode(i))
    ## sub word tokenisation
    ### e.g. likeness, ness is used in alot of words...
    ## there are different ways of tokenisation!



torch.Size([17])
tensor(101) [CLS]
tensor(157) T
tensor(14640) ##hou
tensor(4103) shall
tensor(1136) not
tensor(2561) create
tensor(170) a
tensor(3395) machine
tensor(1107) in
tensor(1103) the
tensor(1176) like
tensor(1757) ##ness
tensor(1104) of
tensor(1103) the
tensor(1769) human
tensor(1713) mind
tensor(102) [SEP]


In [ ]:
# TODO: Encoding multiple texts
message = [
    "Big black bug bleeds black blood",
    "She "
]


In [ ]:
# TODO: Decode text


## Working with LLMs
Create and instance of the Large Language Model (LLM). We will then create a simple
prompt, tokenize the prompt and feed the tokenized prompt to the LLM. The response
from the LLM will be decoded to human friendly text.

In [ ]:
# TODO: Load libraries


In [ ]:
# TODO: Load question answer model



In [ ]:
# TODO: Encode context and question



In [ ]:
# TODO: Tokenize the inputs



In [97]:
# Ensure minimum and maximum token length in the answer
def ensure_size(input_ids, answer, min_length = 2, max_length = 5):
   ans_start = torch.argmax(answer['start_logits'])
   ans_end = torch.argmax(answer['end_logits']) + 1
   ans_length = ans_end - ans_start
   if ans_length < min_length:
      ans_end = min(ans_start + min_length, len(input_ids[0]))
   elif ans_length > max_length:
      ans_end = ans_start + max_length
   ans_start = max(0, ans_start)
   ans_end = min(len(input_ids[0]), ans_end)
   return (ans_start, ans_end)

In [ ]:
# TODO Return a minimum of 5 tokens


In [ ]:
# TODO: Try this your self

context = """
Dickens wrote A Christmas Carol during a period when the British were exploring and re-evaluating past Christmas traditions, 
including carols, and newer customs such as cards and Christmas trees. He was influenced by the experiences of his own youth and 
by the Christmas stories of other authors, including Washington Irving and Douglas Jerrold. Dickens had written three Christmas 
stories prior to the novella, and was inspired following a visit to the Field Lane Ragged School, one of several establishments for 
London's street children. The treatment of the poor and the ability of a selfish man to redeem himself by transforming into a more 
sympathetic character are the key themes of the story. There is discussion among academics as to whether this is a fully secular 
story or a Christian allegory.
"""

question = "How many stories has Dickens wrote?"

